## 安然数据
首先确定目的是为了进行数据分析，因此需要从数据分析的过程去看看怎么样使用 R 来进行相关过程。本次将以安然的部分数据来进行分析，重点将以 EDA 的角度来进行：

1. 读取数据
2. 数据清理
3. **探索分析**
    * 单变量分析
    * 双变量分析
    * 多变量分析
4. 建模

In [14]:
IRkernel::installspec()

In [29]:
install.packages("digest","~/anaconda3/envs//R/lib/")

Warning message in install.packages("digest", "~/anaconda3/envs//R/lib/"):
“installation of package ‘digest’ had non-zero exit status”

In [30]:
install.packages('VIM')

also installing the dependencies ‘zip’, ‘SparseM’, ‘MatrixModels’, ‘openxlsx’, ‘minqa’, ‘nloptr’, ‘RcppEigen’, ‘carData’, ‘pbkrtest’, ‘quantreg’, ‘maptools’, ‘rio’, ‘lme4’, ‘lmtest’, ‘car’, ‘sp’, ‘vcd’, ‘e1071’, ‘laeken’



In [27]:
install.packages("VIM", "~/anaconda3/lib/R/library/")

Warning message in install.packages("VIM", "~/anaconda3/lib/R/library/"):
“installation of package ‘VIM’ had non-zero exit status”

In [ ]:
library(ggplot2)
library(dplyr)

### 读取数据
R 同样可以读取多种类型的数据文件，例如Excel、XML、SPSS、SAS 和 MATLAB，剪贴板，甚至是网络数据。常用的函数中需要注意，`read.csv()` 和 `read.csv2()` 的差异——后者默认是使用`;` 作为分隔符，`,` 作为千分位的 CSV 文件类型。对于其他异常数据作为缺失值，可以通过 `na.strings` 参数进行调整——`na.strings=c("NULL", "#N/A", "#DIV/0")`。

<font size=1 color="green">补充信息：
    <ol>
        <li>如果需要保存相应的数据，那么可以使用 <code>write.table</code> 等函数</li>
        <li>此外如果需要<b>交互</b>，可以使用 <code>readline()</code></li>
        <li>下载数据，可以使用 <code>download.file()</code> 或者 <code>RCurl</code>包获取 HTML 或 XML 数据，而 <code>XML</code> 包中的相关函数可以进行页面解析
    </ol>
</font>

In [1]:
df <- read.csv("./data/enron.csv")

### 清理数据
#### 评估数据
通过相应的函数获取到了数据信息，那么接下来需要了解数据信息以确定接下来的数据清洗的相对应的工作。Python 中 Pandas 提供了很多方便的方式进行数据访问，同样地在 R 中可以使用一些简单的函数来完成相应的工作：

1. 访问 `data.frame` 的前后部分数据，`head()` 与 `tail()`
2. 数据总体信息，例如字段名称以及数据类型 `str()`
3. 字段列名称，`names()`
4. 数据维度信息，`dim()`、`nrow()` 以及 `ncol()`，此外还可以使用 `NROW()` 和 `NCOL()` 来获取向量数据的维度——是将向量作为了一维矩阵
5. 判断数据是否为缺失值，`is.nan()`。此外需要注意 R 是区分缺失值和占位符值——`is.nan()` 和 `is.na()` 检测的是两种情况
6. 行或列方向求和， `rowSums()` 与 `colSums()`

In [3]:
# 使用 head 函数访问前部分数据
head(df, n=2)

Name,bonus,deferral_payments,deferred_income,director_fees,email_address,exercised_stock_options,expenses,from_messages,from_poi_to_this_person,⋯,long_term_incentive,other,poi,restricted_stock,restricted_stock_deferred,salary,shared_receipt_with_poi,to_messages,total_payments,total_stock_value
ALLEN PHILLIP K,4175000,2869717,-3081055,NaN,phillip.allen@enron.com,1729541,13868,2195,47,⋯,304805,152,FALSE,126027,-126027,201955,1407,2902,4484442,1729541
BADUM JAMES P,NaN,178980,NaN,NaN,NaN,257817,3486,NaN,NaN,⋯,NaN,NaN,FALSE,NaN,NaN,NaN,NaN,NaN,182466,257817


In [4]:
# 了解数据整体信息
str(df)

'data.frame':	146 obs. of  22 variables:
 $ Name                     : Factor w/ 146 levels "ALLEN PHILLIP K",..: 1 2 3 4 5 6 7 8 9 10 ...
 $ bonus                    : num  4175000 NaN NaN 1200000 400000 ...
 $ deferral_payments        : num  2869717 178980 NaN 1295738 260455 ...
 $ deferred_income          : num  -3081055 NaN -5104 -1386055 -201641 ...
 $ director_fees            : num  NaN NaN NaN NaN NaN ...
 $ email_address            : Factor w/ 112 levels "NaN","a..martin@enron.com",..: 80 1 30 1 22 1 97 105 1 16 ...
 $ exercised_stock_options  : num  1729541 257817 4046157 6680544 NaN ...
 $ expenses                 : num  13868 3486 56301 11200 129142 ...
 $ from_messages            : num  2195 NaN 29 NaN NaN ...
 $ from_poi_to_this_person  : num  47 NaN 39 NaN NaN NaN 144 228 NaN NaN ...
 $ from_this_person_to_poi  : num  65 NaN 0 NaN NaN NaN 386 108 NaN NaN ...
 $ loan_advances            : num  NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN ...
 $ long_term_incentive      : num  3

In [5]:
# names 函数不仅被用于获取字段名称，而且还可以对列名称进行修改——names(test) <- c("a", "b")
names(df)

[1] "Name"                      "bonus"                    
 [3] "deferral_payments"         "deferred_income"          
 [5] "director_fees"             "email_address"            
 [7] "exercised_stock_options"   "expenses"                 
 [9] "from_messages"             "from_poi_to_this_person"  
[11] "from_this_person_to_poi"   "loan_advances"            
[13] "long_term_incentive"       "other"                    
[15] "poi"                       "restricted_stock"         
[17] "restricted_stock_deferred" "salary"                   
[19] "shared_receipt_with_poi"   "to_messages"              
[21] "total_payments"            "total_stock_value"

In [6]:
# 注意⚠️，此部分作为了解
t <- list(d=c(1, 2, 3))
print(t)
names(t) <- "b"
cat("修改名称后数据：\n")
print(t)

$d
[1] 1 2 3

修改名称后数据：
$b
[1] 1 2 3



以上内容可以帮助了解数据字段相关类型和名称，如果需要了解数据字段的意义，需要了解数据背景信息：
* 财务特征 : [‘salary’, ‘deferral_payments’, ‘total_payments’, ‘loan_advances’, ‘bonus’, ‘restricted_stock_deferred’, ‘deferred_income’, ‘total_stock_value’, ‘expenses’, ‘exercised_stock_options’, ‘other’, ‘long_term_incentive’, ‘restricted_stock’, ‘director_fees’]

* 邮件特征 : [‘to_messages’, ‘email_address’, ‘from_poi_to_this_person’, ‘from_messages’, ‘from_this_person_to_poi’, ‘shared_receipt_with_poi’]

* POI 嫌疑人标签

----
总体来看以上相应的步骤中，可以看出一些相应的信息。首先观察这个数据集的基本信息，有 **146** 条数据，**22** 个变量。这是了解数据的第一步，接下来来看每个字段的字段类型，大部分都是 Num——数值型数据,有一个logi——逻辑型数据，还有factor——因子型数据。查看会发现 factor 数据，email_address 和 Name 是在**读取数据过程中被转换为了 factor 类型；可以通过 `stringAsFactors` 参数进行调整**

如果评估数据其他情况，例如**缺失值**、**异常值**，可以采用其他方式来解决：
1. 缺失值
    除了可以使用 `colSums()`, `rowSums()` 配合 `is.na()` 进行计算意外还可以通过其他 package 来进行处理，例如 `VIM`

In [17]:
# 各字段缺失值情况
colSums(is.na(df))[1:4]

Name             bonus deferral_payments   deferred_income 
                0                64               107                97

In [32]:
dim(df)

[1] 146  22

In [31]:
install.packages("VIM")

Warning message:
“unable to access index for repository https://cran.r-project.org/src/contrib:
  無法載入網際網路例行程式”Warning message:
“package ‘VIM’ is not available (for R version 3.4.2)”

In [30]:
library(purrr)

#### 清理数据